Notes: 
- Set cow_codes to 0 in database.
- Delete `new_societies` that were added
- Verify proper Federal Republic of Central America
- Rerun database updates and insertions.

# Update Societies
## By: Scott Kustes

### Objective:
Update the societies that are already in the database.  Test

#### Contents
<ul>
    <li><a href='#gather'>Data Gathering</a></li>
    <li><a href='#assess1'>Assess, Part 1</a></li>
    <li><a href='#clean1'>Clean, Part 1</a></li>
    <li><a href='#assess2'>Assess, Part 2</a></li>
    <li><a href='#clean2'>Clean, Part 2</a></li>
    <li><a href='#assess3'>Assess, Part 3</a></li>
    <li><a href='#clean3'>Clean, Part 3</a></li>
    <li><a href='#final'>Finished Dataframes</a></li>
    <li><a href='#references'>References</a></li>
</ul>

In [1]:
# Import packages
import requests
import pandas as pd

In [4]:
# Import classes and functions needed for this analysis from config module
# These are only available on my computer
from config import dbaccess, validator, society, error_dict_to_string

# Create an instance of the DBAccess class for running queries
db = dbaccess.DBAccess()

# Create an instance of the Validator class for validating data prior to inserting/updating database
val = validator.Validator()

<a id='gather'></a>
## Data Gathering
Note: Due to making updates and insertions to the `society` database table based on this and other analyses, some of the notes below may no longer reflect reality as it was when this analysis was initially conducted. For instance, after the insertion of `new_societies` into the database, there are no longer any new societies to be inserted.

### Colonization Data

In [39]:
icow = pd.read_csv( 'colonial_data.csv' )
icow.sample(5)

,State,Name,ColRuler,IndFrom,IndDate,IndViol,IndType,SecFrom,SecDate,SecViol,Into,IntoDate,COWsys,GWsys,Notes
195,816,Vietnam/Dem.Rep.Vietnam (North),220,220,195407,1,2,-9,-9,-9,-9,-9,195407,195405,G&W add earlier indep from 1/1/1816-1/1/1893
168,694,Qatar,200,200,197109,0,2,-9,-9,-9,-9,-9,197109,197109,Terr chg also codes as becoming indep in 1915 ...
109,436,Niger,220,220,196008,0,2,-9,-9,-9,-9,-9,196010,196008,Part of French West Africa
48,245,Bavaria,-9,-9,150507,1,1,-9,-9,-9,255,187101,181601,181601,Merged into unified Germany
99,395,Iceland,-9,390,194406,0,3,390,194406,0,-9,-9,194406,194406,-9


### Country Codes
Read in and de-duplicate the country codes used by The Correlates of War Project. Rename columns.

In [9]:
country_codes = pd.read_csv( 'cow_country_codes.csv' )
country_codes.drop_duplicates( inplace=True )
country_codes.rename( columns={'StateAbb': 'abbreviation', 'CCode': 'cow_code', 'StateNme': 'country_name'}, inplace=True )
country_codes.sample(5)

,abbreviation,cow_code,country_name
138,NIG,475,Nigeria
161,NAM,565,Namibia
129,NIR,436,Niger
125,MLI,432,Mali
148,BUI,516,Burundi


Add "Federal Republic of Central America" with `cow_code` of 89. This value is in the `indep_from` column for Guatemala, Honduras, El Salvador, Nicaragua, and Costa Rica, but isn't in the `country_codes` dataset.

In [53]:
frca = {'abbreviation':'FRC', 'cow_code':89, 'country_name': 'Federal Republic of Central America'}
country_codes = country_codes.append(frca, ignore_index=True)

#### Match COW Countries with `society` Table
Many countries already exist in the `society` database table, based on United Nations data. Match COW Project countries with those in the database and execute an UPDATE query to set their `cow_code` field to the corresponding value in the `country_codes` dataframe. Add countries in the COW data that are not in the database to `society`.

In [10]:
# Get societies from the database
query = db.run_query('SELECT society_id, common_name FROM society')
societies = pd.DataFrame.from_dict(query['data'])
societies.sample(5)

,society_id,common_name
220,221,Timor-Leste
141,142,Micronesia
73,74,Fiji
172,173,Pitcairn
224,225,Trinidad and Tobago


Find entries in `country_codes` that are not in the `societies` dataframe (i.e., not in the `society` database table).

In [11]:
print( country_codes[ ~country_codes['country_name'].isin(list(societies['common_name'].unique())) ].shape )
country_codes[ ~country_codes['country_name'].isin(list(societies['common_name'].unique())) ]

(15, 3)


,abbreviation,cow_code,country_name
0,USA,2,United States of America
2,BHM,31,Bahamas
14,SLU,56,St. Lucia
15,SVG,57,St. Vincent and the Grenadines
16,AAB,58,Antigua & Barbuda
17,SKN,60,St. Kitts and Nevis
41,NTH,210,Netherlands
75,CZR,316,Czech Republic
120,CAP,402,Cape Verde
121,STP,403,Sao Tome and Principe


Fifteen of these 40 unmatched countries are already in `societies`, but `country_name` in `country_codes` is different from `common_name` in `societies`. Update `country_name` in `country_codes` dataframe to match `common_name`. Create two dataframes, one for countries already in the database that need to be updated with COW country codes and one for countries that need to be added to the database.

In [19]:
# Update country_name to match common_name in the database
update_cow = {
    'United States of America': 'USA',
    'Bahamas': 'The Bahamas',
    'St. Lucia': 'Saint Lucia',
    'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Antigua & Barbuda': 'Antigua and Barbuda',
    'St. Kitts and Nevis': 'Saint Kitts and Nevis',
    'Netherlands': 'The Netherlands',
    'Czech Republic': 'Czechia',
    'Cape Verde': 'Cabo Verde',
    'Sao Tome and Principe': 'São Tomé and Príncipe',
    'Ivory Coast': 'Côte d’Ivoire',
    'Democratic Republic of the Congo': 'DRC',
    'Swaziland': 'Eswatini',
    'East Timor': 'Timor-Leste',
    'Federated States of Micronesia': 'Micronesia'
}
country_codes.replace({'country_name': update_cow}, inplace=True)

####### Create two dataframes
# Societies already in the database that need to be updated with COW country code
# Use inner merge to get countries with data in database and country_codes dataframe
existing_societies = country_codes.merge( societies, left_on='country_name', right_on='common_name', how='inner' )

# Societies that need to be added to the database
# Find country_name in country_codes dataframe that aren't in existing_societies
new_societies = country_codes[ ~country_codes['country_name'].isin(list(existing_societies['country_name'].unique())) ].copy()

In [28]:
# There were originally 25 new_societies and 192 existing_societies
print( 'New:', new_societies.shape[0] )
print( 'Existing:', existing_societies.shape[0] )

New: 0
Existing: 217


#### Update `society` Table
`1)` Update `society` table in database: Set `cow_code` for records in `existing_societies`.

`2)` Create necessary information for records in `new_societies` and add to `society` table.

##### Update Existing Societies with COW Country Code

In [21]:
# Check if any of the values fall outside of the 0-999 range for COW country codes
# print( "Number of errors:", existing_societies[val.integer_out_of_bounds(existing_societies['cow_code'],0,999)].shape[0] )

Number of errors: 0


In [22]:
# Build an UPDATE statement for each row, then execute them
def build_update_statement(row):
    return 'UPDATE society SET cow_code = ' + str(row['cow_code']) + ' WHERE society_id = ' + str(row['society_id'])

# Create UPDATE statement and execute
existing_societies['update_statement'] = existing_societies.apply(build_update_statement, axis=1)

#records_updated = 0
#row = 1
#print( 'Attempting update of', existing_societies.shape[0], 'rows' )
#for update in existing_societies['update_statement']:
#    update_query = db.run_query( update )
#    records_updated += update_query['rows']
#    print( row, end=" ", flush=True )
#    row += 1

#print( '\n', records_updated, 'rows updated' )

print( 'This has already been run and 192 records were updated.' )

This has already been run and 192 records were updated.


In [23]:
# Pull a few records of society data from the database for a confirmation check
query = db.run_query('SELECT * FROM society ORDER BY RAND() LIMIT 5')
peek = pd.DataFrame.from_dict(query['data'])
peek.sample(5)

,society_id,common_name,official_name,capital,society_type_id,un_m49,iso_alpha3,un_region,cow_code,still_exists
1,58,Côte d’Ivoire,Republic of Côte d'Ivoire,Yamoussoukro,1,384,CIV,011,437,1
4,113,Jordan,Hashemite Kingdom of Jordan,Amman,1,400,JOR,145,663,1
3,63,Dominican Republic,Dominican Republic,Santo Domingo,1,214,DOM,029,42,1
2,27,Bosnia and Herzegovina,Bosnia and Herzegovina,Sarajevo,1,070,BIH,039,346,1
0,101,Iceland,Iceland,Reykjavík,1,352,ISL,154,395,1


##### Add New Societies
Many of these societies had multiple different names and constitutional types (such as duchies, kingdoms, etc). For now, add to the database in barebones format with common name, official name (duplicated from common name), society type ID of 99 ("Uncategorized"), and COW Code. These will be updated later as analysis of political events clarifies things.

This no longer runs because the `new_societies` dataframe is empty after the initial insertion of these societies.

In [29]:
# Drop abbreviation column and rename country_name to common_name
# new_societies.drop(columns=['abbreviation'], inplace=True)
# new_societies.rename(columns={'country_name':'common_name'}, inplace=True)

# Add official_name as duplicate of common_name, and set society_type_id to 99
# new_societies['official_name'] = new_societies['common_name']
# new_societies['society_type_id'] = 99

# new_societies.sample(5)

In [30]:
# Check for errors in the new_societies dataframe before insertion
# print( error_dict_to_string(society.Society.validate_societies(new_societies, db, val)) )

In [31]:
# Insert Societies
#print( 'Expecting to insert:', new_societies.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
#insert_new_societies = db.run_query(db.build_insert_query('society', new_societies))
#print( insert_new_societies['rows'], 'records inserted' )
print( 'This has already been run and 25 records were inserted.' )

This has already been run and 25 records were inserted.


##### Get Societies
Pull society records from the database for use in analysis.

In [32]:
query = db.run_query('SELECT society_id, common_name, cow_code FROM society')
societies = pd.DataFrame.from_dict(query['data'])
societies.sample(5)

,society_id,common_name,cow_code
241,242,Venezuela,101
258,259,Mecklenburg Schwerin,280
217,218,Thailand,800
268,269,Zanzibar,511
24,25,Bhutan,760


<a id='assess1'></a>
## Assess, Part 1

In [33]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 15 columns):
State       217 non-null int64
Name        217 non-null object
ColRuler    217 non-null int64
IndFrom     217 non-null int64
IndDate     217 non-null int64
IndViol     217 non-null int64
IndType     217 non-null int64
SecFrom     217 non-null int64
SecDate     217 non-null int64
SecViol     217 non-null int64
Into        217 non-null int64
IntoDate    217 non-null int64
COWsys      217 non-null int64
GWsys       217 non-null int64
Notes       217 non-null object
dtypes: int64(13), object(2)
memory usage: 25.6+ KB


In [34]:
icow.sample(5)

,State,Name,ColRuler,IndFrom,IndDate,IndViol,IndType,SecFrom,SecDate,SecViol,Into,IntoDate,COWsys,GWsys,Notes
13,58,Antigua and Barbuda,200,200,198111,0,2,-9,-9,-9,-9,-9,198111,-9,Includes Antigua/Barbuda/Redonda islands
193,811,Cambodia,220,220,195311,1,2,-9,-9,-9,-9,-9,195311,195311,Ruled by Siam before French acquisition in 1863
66,327,Papal States,-9,-9,150300,1,1,-9,-9,-9,325,186011,181601,181601,Merged into unified Italy
48,245,Bavaria,-9,-9,150507,1,1,-9,-9,-9,255,187101,181601,181601,Merged into unified Germany
185,760,Bhutan,200,750,194708,0,2,-9,-9,-9,-9,-9,197109,194901,Ruled by Chinese before British/Indians; Briti...


In [35]:
# Unique values in Colonial Ruler column
print( icow['ColRuler'].nunique() )
icow['ColRuler'].unique()

17


array([200, 230, 220, 210, 235,  -9, 255, 300, 640, 365, 211, 325, 710,
       740, 900,   2, 920], dtype=int64)

In [36]:
icow.query( '( ColRuler == -9 ) & ( IndFrom == -9 ) & ( SecFrom == -9 ) & ( Into == -9 )' ).sample(5)

,State,Name,ColRuler,IndFrom,IndDate,IndViol,IndType,SecFrom,SecDate,SecViol,Into,IntoDate,COWsys,GWsys,Notes
96,380,Sweden,-9,-9,152306,1,4,-9,-9,-9,-9,-9,181601,181601,Broke away from Union of Kalmar (with Denmark/...
43,225,Switzerland,-9,-9,129108,0,1,-9,-9,-9,-9,-9,181601,181601,-9
183,740,Japan,-9,-9,159000,1,1,-9,-9,-9,-9,-9,186001,181601,COW system interrupted 8/1945-4/1952 (Allied o...
45,232,Andorra,-9,-9,127809,0,1,-9,-9,-9,-9,-9,199307,-9,Nominal French & Spanish suzerainty not enough...
35,200,United Kingdom,-9,-9,106612,1,1,-9,-9,-9,-9,-9,181601,181601,-9


### Issues Found
`1)` Rename columns to more reader-friendly format.

`2)` Remove countries with -9 in `ColRuler`, `IndFrom`, `SecFrom`, and `Into`. Save them into another dataframe in case there is useful information at a later time.

<a id='clean1'></a>
## Clean, Part 1

### 1) Rename Columns
Create reader-friendly column names.

#### Code

In [40]:
icow.rename( columns={
    'State': 'cow_code',
    'Name': 'country_name',
    'ColRuler': 'colonizer',
    'IndFrom': 'indep_from',
    'IndDate': 'indep_date',
    'IndViol': 'indep_violent',
    'IndType': 'indep_type',
    'SecFrom': 'secession_from',
    'SecDate': 'secession_date',
    'SecViol': 'secession_violent',
    'Into': 'merged_into',
    'IntoDate': 'merged_date',
    'COWsys': 'cow_indep_date',
    'GWsys': 'gw_indep_date',
    'Notes': 'notes'
}, inplace=True )

#### Test

In [41]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 15 columns):
cow_code             217 non-null int64
country_name         217 non-null object
colonizer            217 non-null int64
indep_from           217 non-null int64
indep_date           217 non-null int64
indep_violent        217 non-null int64
indep_type           217 non-null int64
secession_from       217 non-null int64
secession_date       217 non-null int64
secession_violent    217 non-null int64
merged_into          217 non-null int64
merged_date          217 non-null int64
cow_indep_date       217 non-null int64
gw_indep_date        217 non-null int64
notes                217 non-null object
dtypes: int64(13), object(2)
memory usage: 25.6+ KB


### 2) Remove Uncolonized Countries
Delete all entries where `ColRuler`, `IndFrom`, `SecFrom`, and `Into` columns are -9. These countries were never colonized, never declared independence or seceded from another country, and never merged into another country and are therefore unnecessary in the dataset.

#### Code

In [43]:
always_independent = icow.query( '( colonizer == -9 ) & ( indep_from == -9 ) & ( secession_from == -9 ) & ( merged_into == -9 )' ).copy()
icow.drop( icow.query( '( colonizer == -9 ) & ( indep_from == -9 ) & ( secession_from == -9 ) & ( merged_into == -9 )' ).index, inplace=True )

#### Test

In [44]:
icow.shape

(193, 15)

In [45]:
always_independent.shape

(24, 15)

In [46]:
always_independent.sample(5)

,cow_code,country_name,colonizer,indep_from,indep_date,indep_violent,indep_type,secession_from,secession_date,secession_violent,merged_into,merged_date,cow_indep_date,gw_indep_date,notes
113,450,Liberia,-9,-9,184707,0,1,-9,-9,-9,-9,-9,192006,184707,-9
45,232,Andorra,-9,-9,127809,0,1,-9,-9,-9,-9,-9,199307,-9,Nominal French & Spanish suzerainty not enough...
46,235,Portugal,-9,-9,113907,0,1,-9,-9,-9,-9,-9,181601,181601,Dependency of Spain 1580-1640
35,200,United Kingdom,-9,-9,106612,1,1,-9,-9,-9,-9,-9,181601,181601,-9
154,630,Iran (Persia),-9,-9,150200,1,1,-9,-9,-9,-9,-9,185501,181601,-9


<a id='assess2'></a>
## Assess, Part 2

# Start Here

In [47]:
print( icow['colonizer'].nunique() )
icow['colonizer'].unique()

17


array([200, 230, 220, 210, 235,  -9, 255, 300, 640, 365, 211, 325, 710,
       740, 900,   2, 920], dtype=int64)

In [48]:
print( icow['indep_from'].nunique() )
icow['indep_from'].unique()

34


array([200,   2, 220,  41, 230,  89, 100, 210, 235, 140,  -9, 255, 300,
       315, 640, 345, 365, 380, 390, 432, 211, 325, 530, 560, 625, 678,
       710, 730, 750, 770, 820, 850, 900, 920], dtype=int64)

### Issues Found
`1)` Add country name into dataframe for columns `colonizer`, `indep_from`, `secession_from`, and `merged_into`.

`2)` Convert `indep_date`, `cow_indep_date`, and `gw_indep_date` from int to Time Period.

<a id='clean2'></a>
## Clean, Part 2

### 1) Add Country Name for columns with Country Codes
Insert columns to hold country name based on country codes in `colonizer`, `indep_from`, `secession_from`, and `merged_into` columns. 

Five Central American countries were part of the Federal Republic of Central America (country code: 89), which is not in the Correlates of War Project list. Manually update these entries. Morocco and Saudi Arabia will remain as nan because indep_from is -9.

#### Code

In [ ]:
columns = ['colonizer','indep_from','secession_from','merged_into']
for column in columns:
    # Join on country code, then rename the joined column
    icow = icow.join( country_codes['country_name'], on=column )
    new_column_name = column + '_name'
    icow.rename( columns={'country_name': new_column_name}, inplace=True )

In [ ]:
indexes = icow.query( 'indep_from == 89' ).index.tolist()
icow.loc[ indexes, 'indep_from_name' ] = 'Federal Republic of Central America'

#### Test

In [ ]:
icow.info()

In [ ]:
print( icow['colonizer'].nunique() )
print( icow['colonizer_name'].nunique() )
icow['colonizer_name'].unique()

In [ ]:
print( icow['indep_from'].nunique() )
print( icow['indep_from_name'].nunique() )
icow['indep_from_name'].unique()

In [ ]:
icow[ icow['indep_from_name'].isnull() ]

### 2) Convert date columns to datetime

Convert `indep_date`, `secession_date`, `merged_date`, `cow_indep_date`, and `gw_indep_date` from int to Time Period. 

**Note to future me**: I originally attempted a conversion to datetime, but datetime has a limitation to dates between 1677-09-21 and 2262-04-11. A few values fall outside of this range, therefore the workaround found <a href='https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-oob' target='_new'>here</a> was used. This caused the following problem: <a href='https://stackoverflow.com/questions/58019763/jupyter-kernel-crash-when-querying-dataframe-with-period-datatype' target='_new'>Jupyter kernel crash when querying dataframe with Period datatype</a>. 

*Solution:* The error was caused by -9 in the `gw_indep_date` column. While the conversion to Period didn't fail, it somehow corrupted the dataframe, making queries impossible. By setting -9 dates to 000101 prior to conversion, a valid Period of 0001-01-01 is obtained and the dataframe functions properly.

#### Code

In [ ]:
# Conversion function to create Time Period, copied without modification from Pandas documentation linked in the cell above
def conv(x):
    return pd.Period( year=x // 10000, month=x // 100 % 100, day=x % 100, freq='D' )

# -9 entries in date columns cause problems with the dataframe after conversion to Period
# This sets them to a valid far past date of 000101
def fix_dates(x):
    return '000101' if x == -9 else x

In [ ]:
# Convert string representations of date to Time Period
# First, apply fix_dates function to each column to ensure missing data converts properly (-9 in the original dataset)
# Then append '01' to create format YYYYMMDD (dataset contains only YYYYMM), then apply conversion function
columns = ['indep_date','secession_date','merged_date','cow_indep_date','gw_indep_date']
for column in columns:
    icow[column] = icow[column].apply(fix_dates)
    icow[column] = ( icow[column].astype('str') + '01' ).astype('int').apply(conv)

#### Test

In [ ]:
icow.info()

In [ ]:
# Check the three entries that failed when attempting conversion to datetime
icow[ icow.index.isin( [148,150,170] ) ][['country','indep_date','cow_indep_date','gw_indep_date']]

<a id='assess3'></a>
## Assess, Part 3

In [ ]:
icow['colonizer'].value_counts()

In [ ]:
icow['indep_from'].value_counts()

In [ ]:
icow['secession_from'].value_counts()

In [ ]:
icow['merged_into'].value_counts()

### Issues Found
`1)` Numerous -9 values found in `colonizer`, `indep_from`, `secession_from`, and `merged_into` columns. Split into 4 datasets, one dataset for each of the four columns where the value is not -9. 

<a id='clean3'></a>
## Clean, Part 3

### 1) Split into 4 datasets
Create a dataset each for colonized countries, countries that declared independence, countries that seceded, and countries that merged into another. To populate these datasets, get all values that are not -9 from the corresponding columns: `colonizer`, `indep_from`, `secession_from`, `merged_into`

#### Code

In [ ]:
colonized = icow.query( 'colonizer != -9' ).copy()
colonized.shape

In [ ]:
independence = icow.query( 'indep_from != -9' ).copy()
independence.shape

In [ ]:
seceded = icow.query( 'secession_from != -9' ).copy()
seceded.shape

In [ ]:
merged = icow.query( 'merged_into != -9' ).copy()
merged.shape

#### Test

In [ ]:
colonized['colonizer'].value_counts()

In [ ]:
independence['indep_from'].value_counts()

In [ ]:
seceded['secession_from'].value_counts()

In [ ]:
merged['merged_into'].value_counts()

<a id='references'></a>
## References

<li>Paul R. Hensel (2018). "ICOW Colonial History Data Set, version 1.1." Available at <a href='http://www.paulhensel.org/icowcol.html' target='_new'>http://www.paulhensel.org/icowcol.html</a></li>
<li><a href='http://www.correlatesofwar.org/data-sets/downloadable-files/cow-country-codes' target='_new'>Correlates of War country codes</a></li>